## 메시지 기록(메모리) 추가하기

RunnableWithMessageHistory를 활용하여 특정 유형의 작업(체인)에 메시지 기록을 추가하는 것은 프로그래밍에서 상당히 유용한 기능입니다.

이 기능은 특히 대화형 애플리케이션 또는 복잡한 데이터 처리 작업을 구현할 때 이전 메시지의 맥락을 유지해야 할 필요가 있을 때 중요합니다.

메시지 기록을 관리함으로써, 개발자는 애플리케이션의 흐름을 더 잘 제어하고, 사용자의 이전 요청에 따라 적절하게 응답할 수 있습니다.

실제 활용 예시

- 대화형 챗봇 개발: 사용자와의 대화 내역을 기반으로 챗봇의 응답을 조정할 수 있습니다.
- 복잡한 데이터 처리: 데이터 처리 과정에서 이전 단계의 결과를 참조하여 다음 단계의 로직을 결정할 수 있습니다.
- 상태 관리가 필요한 애플리케이션: 사용자의 이전 선택을 기억하고 그에 따라 다음 화면이나 정보를 제공할 수 있습니다.

RunnableWithMessageHistory는 애플리케이션의 상태를 유지하고, 사용자 경험을 향상시키며, 더 정교한 응답 메커니즘을 구현할 수 있게 해주는 강력한 도구입니다.

### 튜토리얼

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 {ability} 에 능숙한 어시스턴트입니다. 20자 이내로 응답하세요",
        ),
        # 대화 기록을 변수로 사용, history 가 MessageHistory 의 key 가 됨
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),  # 사용자 입력을 변수로 사용
    ]
)
runnable = prompt | model  # 프롬프트와 모델을 연결하여 runnable 객체 생성

1. 인메모리 ChatMessageHistory 사용

- 이 방법은 메모리 내에서 메시지 기록을 관리합니다. 주로 개발 단계나 간단한 애플리케이션에서 사용됩니다. 인메모리 방식은 빠른 접근 속도를 제공하지만, 애플리케이션을 재시작할 때 메시지 기록이 사라지는 단점이 있습니다.

2. RedisChatMessageHistory를 사용하여 영구적인 저장소 활용

- Redis를 사용하는 방법은 메시지 기록을 영구적으로 저장할 수 있게 해줍니다. Redis는 높은 성능을 제공하는 오픈 소스 인메모리 데이터 구조 저장소로, 분산 환경에서도 안정적으로 메시지 기록을 관리할 수 있습니다. 이 방법은 복잡한 애플리케이션 또는 장기간 운영되는 서비스에 적합합니다.

### 휘발성 대화기록: 인메모리(In-Memory)

RunnableWithMessageHistory 설정 매개변수

- runnable
- BaseChatMessageHistory 이거나 상속받은 객체. ex) ChatMessageHistory
- input_messages_key: chain 을 invoke() 할때 사용자 쿼리 입력으로 지정하는 key
- history_messages_key: 대화 기록으로 지정하는 key

In [4]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}  # 세션 기록을 저장할 딕셔너리


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids: str) -> BaseChatMessageHistory:
    print(session_ids)
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


with_message_history = (
    RunnableWithMessageHistory(  # RunnableWithMessageHistory 객체 생성
        runnable,  # 실행할 Runnable 객체
        get_session_history,  # 세션 기록을 가져오는 함수
        input_messages_key="input",  # 입력 메시지의 키
        history_messages_key="history",  # 기록 메시지의 키
    )
)

In [5]:
with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {"ability": "math", "input": "What does cosine mean?"},
    # 설정 정보로 세션 ID "abc123"을 전달합니다.
    config={"configurable": {"session_id": "abc123"}},
)

abc123


AIMessage(content='Ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 47, 'total_tokens': 62}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-50c8ab36-c290-4267-8249-46bed38c5d95-0', usage_metadata={'input_tokens': 47, 'output_tokens': 15, 'total_tokens': 62})

In [6]:
# 메시지 기록을 포함하여 호출합니다.
with_message_history.invoke(
    # 능력과 입력을 설정합니다.
    {"ability": "math", "input": "이전의 내용을 한글로 답변해 주세요."},
    # 설정 옵션을 지정합니다.
    config={"configurable": {"session_id": "abc123"}},
)

abc123


AIMessage(content='직각 삼각형에서 인접한 변의 길이를 빗변의 길이로 나눈 값입니다.', response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 87, 'total_tokens': 126}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-859c771e-3cf0-4ca0-9514-3b6bc98cef7c-0', usage_metadata={'input_tokens': 87, 'output_tokens': 39, 'total_tokens': 126})

In [7]:
# 새로운 session_id로 인해 이전 대화 내용을 기억하지 않습니다.
with_message_history.invoke(
    # 수학 능력과 입력 메시지를 전달합니다.
    {"ability": "math", "input": "이전의 내용을 한글로 답변해 주세요"},
    # 새로운 session_id를 설정합니다.
    config={"configurable": {"session_id": "def234"}},
)

def234


AIMessage(content='수학에 능숙한 어시스턴트입니다.', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 58, 'total_tokens': 77}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1a6101ba-169b-4173-89fb-deb67e0ad1c4-0', usage_metadata={'input_tokens': 58, 'output_tokens': 19, 'total_tokens': 77})

In [8]:
from langchain_core.runnables import ConfigurableFieldSpec

store = {}  # 빈 딕셔너리를 초기화합니다.


def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    # 주어진 user_id와 conversation_id에 해당하는 세션 기록을 반환합니다.
    if (user_id, conversation_id) not in store:
        # 해당 키가 store에 없으면 새로운 ChatMessageHistory를 생성하여 저장합니다.
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[  # 기존의 "session_id" 설정을 대체하게 됩니다.
        ConfigurableFieldSpec(
            id="user_id",  # get_session_history 함수의 첫 번째 인자로 사용됩니다.
            annotation=str,
            name="User ID",
            description="사용자의 고유 식별자입니다.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",  # get_session_history 함수의 두 번째 인자로 사용됩니다.
            annotation=str,
            name="Conversation ID",
            description="대화의 고유 식별자입니다.",
            default="",
            is_shared=True,
        ),
    ],
)

In [9]:
with_message_history.invoke(
    # 능력(ability)과 입력(input)을 포함한 딕셔너리를 전달합니다.
    {"ability": "math", "input": "what is cosine?"},
    # 설정(config) 딕셔너리를 전달합니다.
    config={"configurable": {"user_id": "123", "conversation_id": "abc"}},
)

AIMessage(content='cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 46, 'total_tokens': 72}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-638ae066-6453-4770-87c7-15ca44dd3590-0', usage_metadata={'input_tokens': 46, 'output_tokens': 26, 'total_tokens': 72})

In [10]:
answer = with_message_history.invoke(
    # 능력(ability)과 입력(input)을 포함한 딕셔너리를 전달합니다.
    {"ability": "math", "input": "이전의 답변을 한글로 작성해 주세요"},
    # 설정(config) 딕셔너리를 전달합니다.
    config={"configurable": {"user_id": "123", "conversation_id": "abc"}},
)

### 다양한 Key를 사용한 Runnable 을 사용한 예시

#### Messages 객체를 입력, dict 형태의 출력

In [11]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

# chain 생성
chain = RunnableParallel({"output_message": ChatOpenAI()})


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    # 세션 ID에 해당하는 대화 기록이 저장소에 없으면 새로운 ChatMessageHistory를 생성합니다.
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    # 세션 ID에 해당하는 대화 기록을 반환합니다.
    return store[session_id]


# 체인에 대화 기록 기능을 추가한 RunnableWithMessageHistory 객체를 생성합니다.
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    # 입력 메시지의 키를 "input"으로 설정합니다.(생략시 Message 객체로 입력)
    # input_messages_key="input",
    # 출력 메시지의 키를 "output_message"로 설정합니다. (생략시 Message 객체로 출력)
    output_messages_key="output_message",
)

# 주어진 메시지와 설정으로 체인을 실행합니다.
with_message_history.invoke(
    # 혹은 "what is the definition of cosine?" 도 가능
    [HumanMessage(content="what is the definition of cosine?")],
    config={"configurable": {"session_id": "abc123"}},
)

{'output_message': AIMessage(content='Cosine is a trigonometric function that relates the length of the adjacent side of a right triangle to the length of the hypotenuse. It is defined as the ratio of the length of the adjacent side to the length of the hypotenuse in a right triangle. The cosine function is commonly denoted as cos(θ), where θ is the angle between the adjacent side and the hypotenuse.', response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 14, 'total_tokens': 97}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6b0b0659-13d2-4869-999c-7051f3ee1e56-0', usage_metadata={'input_tokens': 14, 'output_tokens': 83, 'total_tokens': 97})}

In [12]:
with_message_history.invoke(
    # 이전의 답변에 대하여 한글로 답변을 재요청합니다.
    [HumanMessage(content="이전의 내용을 한글로 답변해 주세요!")],
    # 설정 옵션을 딕셔너리 형태로 전달합니다.
    config={"configurable": {"session_id": "abc123"}},
)

{'output_message': AIMessage(content='코사인은 직각삼각형의 인접 변의 길이와 빗변의 길이를 관련시키는 삼각함수입니다. 코사인은 인접 변의 길이를 빗변의 길이로 나눈 비율로 정의되며, 직각삼각형에서 인접 변과 빗변 사이의 각도를 나타냅니다. 코사인 함수는 일반적으로 cos(θ)로 표시되며, 여기서 θ는 인접 변과 빗변 사이의 각을 의미합니다.', response_metadata={'token_usage': {'completion_tokens': 175, 'prompt_tokens': 122, 'total_tokens': 297}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8b5cfa91-c8f0-4785-a385-6cf06abe8a3a-0', usage_metadata={'input_tokens': 122, 'output_tokens': 175, 'total_tokens': 297})}

#### Messages 객체를 입력, Messages 객체의 출력

In [13]:
with_message_history = RunnableWithMessageHistory(
    ChatOpenAI(),  # ChatOpenAI 언어 모델을 사용합니다.
    get_session_history,  # 대화 세션 기록을 가져오는 함수를 지정합니다.
    # 입력 메시지의 키를 "input"으로 설정합니다.(생략시 Message 객체로 입력)
    # input_messages_key="input",
    # 출력 메시지의 키를 "output_message"로 설정합니다. (생략시 Message 객체로 출력)
    # output_messages_key="output_message",
)

In [14]:
with_message_history.invoke(
    # 이전의 답변에 대하여 한글로 답변을 재요청합니다.
    [HumanMessage(content="코사인의 의미는 무엇인가요?")],
    # 설정 옵션을 딕셔너리 형태로 전달합니다.
    config={"configurable": {"session_id": "def123"}},
)

AIMessage(content='코사인은 삼각함수 중 하나로, 직각삼각형에서 빗변과 인접 변의 길이를 이용하여 각도를 구하는 함수입니다. 각도 θ에 대해 코사인은 삼각형의 밑변과 빗변의 비율을 나타내며, 수학적으로는 코사인 함수는 삼각형의 밑변을 빗변으로 나눈 값으로 정의됩니다. 즉, 코사인은 각도에 대한 삼각비를 나타내며, 주어진 각도에 대한 코사인 값을 구하는 것은 주어진 삼각형에서의 밑변과 빗변의 길이를 이용하여 각도를 계산하는 것과 동일합니다.', response_metadata={'token_usage': {'completion_tokens': 231, 'prompt_tokens': 24, 'total_tokens': 255}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7fbdc333-20ae-41af-9007-a928e74df98f-0', usage_metadata={'input_tokens': 24, 'output_tokens': 231, 'total_tokens': 255})

#### 모든 메시지 입력과 출력을 위한 단일 키를 가진 Dict

In [15]:
from operator import itemgetter

with_message_history = RunnableWithMessageHistory(
    # "input_messages" 키를 사용하여 입력 메시지를 가져와 ChatOpenAI()에 전달합니다.
    itemgetter("input_messages") | ChatOpenAI(),
    get_session_history,  # 세션 기록을 가져오는 함수입니다.
    input_messages_key="input_messages",  # 입력 메시지의 키를 지정합니다.
)

In [16]:
with_message_history.invoke(
    {"input_messages": "코사인의 의미는 무엇인가요?"},
    # 설정 옵션을 딕셔너리 형태로 전달합니다.
    config={"configurable": {"session_id": "xyz123"}},
)

AIMessage(content='코사인(cosine)은 삼각함수 중 하나로, 직각삼각형에서의 각도에 대한 값을 나타냅니다. 코사인은 한 삼각형의 밑변과 빗변의 길이 비율을 나타내며, 각도가 0도일 때 1이 되고 90도일 때 0이 됩니다. 즉, 코사인 값은 -1부터 1까지의 값을 가집니다. 코사인은 삼각형의 각도나 길이를 계산하는데 사용되며, 수학, 공학, 물리학 등 다양한 분야에서 활용됩니다.', response_metadata={'token_usage': {'completion_tokens': 198, 'prompt_tokens': 24, 'total_tokens': 222}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a624ce39-fcd5-4d90-9fa2-492b5ace9b3e-0', usage_metadata={'input_tokens': 24, 'output_tokens': 198, 'total_tokens': 222})

### 영구 저장소(Persistent storage)

- RunnableWithMessageHistory는 get_session_history 호출 가능 객체가 채팅 메시지 기록을 어떻게 검색하는지에 대해 독립적입니다.
- 로컬 파일 시스템을 사용하는 예제는 여기를 참조하세요.
- 아래에서는 Redis 를 사용하는 방법을 보여줍니다. 다른 제공자를 사용하여 채팅 메시지 기록을 구현하는 방법은 memory integrations 페이지를 확인하세요.

### Redis 설치

In [17]:
# !pip install -qU redis

### Redis 서버 구동

In [18]:
# Redis 서버의 URL을 지정합니다.
REDIS_URL = "redis://localhost:6379/0"

In [19]:
from dotenv import load_dotenv
import os

load_dotenv()

# LANGCHAIN_TRACING_V2 환경 변수를 "true"로 설정합니다.
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# LANGCHAIN_PROJECT 설정
os.environ["LANGCHAIN_PROJECT"] = "RunnableWithMessageHistory"

In [20]:
from langchain_community.chat_message_histories import RedisChatMessageHistory


def get_message_history(session_id: str) -> RedisChatMessageHistory:
    # 세션 ID를 기반으로 RedisChatMessageHistory 객체를 반환합니다.
    return RedisChatMessageHistory(session_id, url=REDIS_URL)


with_message_history = RunnableWithMessageHistory(
    runnable,  # 실행 가능한 객체
    get_message_history,  # 메시지 기록을 가져오는 함수
    input_messages_key="input",  # 입력 메시지의 키
    history_messages_key="history",  # 기록 메시지의 키
)

In [22]:
with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {"ability": "math", "input": "What does cosine mean?"},
    # 설정 옵션으로 세션 ID를 "redis123" 로 지정합니다.
    config={"configurable": {"session_id": "redis123"}},
)
# 도커 관련 문제로 실행 실패

In [ ]:
with_message_history.invoke(
    # 이전 답변에 대한 한글 번역을 요청합니다.
    {"ability": "math", "input": "이전의 답변을 한글로 번역해 주세요."},
    # 설정 값으로 세션 ID를 "foobar"로 지정합니다.
    config={"configurable": {"session_id": "redis123"}},
)

In [ ]:
with_message_history.invoke(
    # 이전 답변에 대한 한글 번역을 요청합니다.
    {"ability": "math", "input": "이전의 답변을 한글로 번역해 주세요."},
    # 설정 값으로 세션 ID를 "redis456"로 지정합니다.
    config={"configurable": {"session_id": "redis456"}},
)